In [2]:
# Starting Point:
# Evolving Simple Organisms using a Genetic Algorithm and Deep Learning from Scratch with Python
# https://nathanrooy.github.io/posts/2017-11-30/evolving-simple-organisms-using-a-genetic-algorithm-and-deep-learning/

import operator
from collections import defaultdict
from math import atan2, cos, degrees, floor, radians, sin, sqrt
from random import randint, random, sample, uniform

import matplotlib.lines as lines
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

In [70]:
settings = {}

# EVOLUTION SETTINGS
settings["pop_size"] = 50  # number of organisms
settings["food_num"] = 100  # number of food particles
settings["gens"] = 100  # number of generations
settings["elitism"] = 0.20  # elitism (selection bias)
settings["mutate"] = 0.10  # mutation rate

# SIMULATION SETTINGS
settings["gen_time"] = 120  # generation length         (seconds)
settings["dt"] = 0.04  # simulation time step      (dt)
settings["dr_max"] = 720  # max rotational speed      (degrees per second)
settings["v_max"] = 0.5  # max velocity              (units per second)
settings["dv_max"] = 0.25  # max acceleration (+/-)    (units per second^2)

settings["x_min"] = -2.0  # arena western border
settings["x_max"] = 2.0  # arena eastern border
settings["y_min"] = -2.0  # arena southern border
settings["y_max"] = 2.0  # arena northern border

settings["plot"] = True  # plot final generation

# ORGANISM NEURAL NET SETTINGS
settings["inodes"] = 1  # number of input nodes
settings["hnodes"] = 5  # number of hidden nodes
settings["onodes"] = 2  # number of output nodes

In [5]:
class Organism:
    def __init__(self, settings, wih=None, who=None, name=None):

        self.x = np.random.uniform(settings["x_min"], settings["x_max"])  # x position
        self.y = np.random.uniform(settings["y_min"], settings["y_max"])  # y position

        self.r = np.random.uniform(0, 360)  # orientation (degrees)
        self.v = np.random.uniform(0, settings["v_max"])  # velocity
        self.dv = np.random.uniform(
            -settings["dv_max"], settings["dv_max"]
        )  # acceleration

        self.d_food = 100  # distance to nearest food
        self.r_food = 0  # direction to nearest food
        self.fitness = 0  # fitness (food count)

        self.wih = wih  # Input layer weights
        self.who = who  # hidden+output layer weights

        self.name = name

    # Neural net
    def think(self):

        # Simple MLP
        af = lambda x: np.tanh(x)  # activation function, tanh
        h1 = af(np.dot(self.wih, self.r_food))  # hidden layer
        out = af(np.dot(self.who, h1))  # output layer

        # Update dv and dr with MLP response
        self.nn_dv = float(out[0])  # [-1, 1], [decelerate, accelerate]
        self.nn_dr = float(out[1])  # [-1, 1], [right, left]

    # Update heading
    def update_r(self, settings):
        self.r += self.nn_dr * settings["dr_max"] * settings["dt"]
        self.r = self.r % 360

    # Update velocity
    def update_v(self, settings):
        self.v += self.nn_dv * settings["dv_max"] * settings["dt"]

        if self.v < 0:
            self.v = 0
        elif self.v > settings["v_max"]:
            self.v = settings["v_max"]

    # Update position
    def update_pos(self, settings):
        dx = self.v * cos(radians(self.r)) * settings["dt"]
        dy = self.v * sin(radians(self.r)) * settings["dt"]

        self.x += dx
        self.y += dy

In [6]:
class Food:
    def __init__(self, settings):
        self.x = np.random.uniform(settings["x_min"], settings["x_max"])  # x position
        self.y = np.random.uniform(settings["y_min"], settings["y_max"])  # y position
        self.energy = 1

    def respawn(self, settings):
        self.__init__(settings)

In [44]:
def evolve(settings, organisms_old, gen):
    def mutate(value):
        """Modify a value by up to 10% while restricting to [-1, 1]"""
        value *= uniform(0.9, 1.1)
        if value > 1:
            value = 1
        elif value < -1:
            value = -1
        return value

    elitism_num = int(floor(settings["elitism"] * settings["pop_size"]))
    new_orgs = settings["pop_size"] - elitism_num

    # Get stats from current gen
    stats = defaultdict(int)
    for org in organisms_old:
        if org.fitness > stats["BEST"] or stats["BEST"] == 0:
            stats["BEST"] = org.fitness

        if org.fitness < stats["WORST"] or stats["WORST"] == 0:
            stats["WORST"] = org.fitness

        stats["SUM"] += org.fitness
        stats["COUNT"] += 1

    stats["AVG"] = stats["SUM"] / stats["COUNT"]

    # Elitism
    old_orgs_by_fitness = sorted(
        organisms_old, key=operator.attrgetter("fitness"), reverse=True
    )
    organisms_new = []
    for i, org in enumerate(old_orgs_by_fitness):
        if i == elitism_num:
            break
        organisms_new.append(
            Organism(settings, wih=org.wih, who=org.who, name=org.name)
        )

    # Generate new organisms

    for i in range(new_orgs):

        # Selection (truncation selection)
        candidates = range(elitism_num)
        parent_indices = sample(candidates, 2)
        parent_1 = old_orgs_by_fitness[parent_indices[0]]
        parent_2 = old_orgs_by_fitness[parent_indices[1]]

        # Crossover
        crossover_weight = random()
        wih_new = (crossover_weight * parent_1.wih) + (
            (1 - crossover_weight) * parent_2.wih
        )
        who_new = (crossover_weight * parent_1.who) + (
            (1 - crossover_weight) * parent_2.who
        )

        # Mutation
        mutate_chance = random()
        if mutate_chance <= settings["mutate"]:

            # Pick mutation matrix
            mat = randint(0, 1)

            # Mutate hidden weight
            if mat == 0:
                mutation_site = randint(0, settings["hnodes"] - 1)
                wih_new[mutation_site] = mutate(wih_new[mutation_site])

            # Mutate output weight
            elif mat == 1:
                mutation_row = randint(0, settings["onodes"] - 1)
                mutation_col = randint(0, settings["hnodes"] - 1)
                who_new[mutation_row][mutation_col] = mutate(
                    who_new[mutation_row][mutation_col]
                )

        organisms_new.append(
            Organism(
                settings,
                wih=wih_new,
                who=who_new,
                name="gen[" + str(gen) + "]-org[" + str(i) + "]",
            )
        )

    return organisms_new, stats

In [62]:
def plot_organism(x1, y1, theta, ax):

    circle = Circle([x1, y1], 0.05, edgecolor="g", facecolor="lightgreen", zorder=8)
    ax.add_artist(circle)

    edge = Circle([x1, y1], 0.05, facecolor="None", edgecolor="darkgreen", zorder=8)
    ax.add_artist(edge)

    tail_len = 0.075

    x2 = cos(radians(theta)) * tail_len + x1
    y2 = sin(radians(theta)) * tail_len + y1

    ax.add_line(
        lines.Line2D([x1, x2], [y1, y2], color="darkgreen", linewidth=1, zorder=10)
    )


def plot_food(x1, y1, ax):

    circle = Circle(
        [x1, y1], 0.03, edgecolor="darkslateblue", facecolor="mediumslateblue", zorder=5
    )
    ax.add_artist(circle)


def dist(x1, y1, x2, y2):
    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def calc_heading(org, food):
    d_x = food.x - org.x
    d_y = food.y - org.y
    theta_d = degrees(atan2(d_y, d_x)) - org.r
    if abs(theta_d) > 180:
        theta_d += 360
    return theta_d / 180


def plot_frame(settings, organisms, foods, gen, time):
    fig, ax = plt.subplots()
    fig.set_size_inches(9.6, 5.4)

    plt.xlim(
        [
            settings["x_min"] + settings["x_min"] * 0.25,
            settings["x_max"] + settings["x_max"] * 0.25,
        ]
    )
    plt.ylim(
        [
            settings["y_min"] + settings["y_min"] * 0.25,
            settings["y_max"] + settings["y_max"] * 0.25,
        ]
    )

    # PLOT ORGANISMS
    for organism in organisms:
        plot_organism(organism.x, organism.y, organism.r, ax)

    # PLOT FOOD PARTICLES
    for food in foods:
        plot_food(food.x, food.y, ax)

    # MISC PLOT SETTINGS
    ax.set_aspect("equal")
    frame = plt.gca()
    frame.axes.get_xaxis().set_ticks([])
    frame.axes.get_yaxis().set_ticks([])

    plt.figtext(0.025, 0.95, r"GENERATION: " + str(gen))
    plt.figtext(0.025, 0.90, r"T_STEP: " + str(time))

    plt.savefig(f"figs/{str(gen)}-{str(time)}.png", dpi=100)
    #plt.show()

In [55]:
def simulate(settings, organisms, foods, gen):

    total_time_steps = int(settings["gen_time"] / settings["dt"])

    for step in range(total_time_steps):
        # Plot frame
        if gen == settings["gens"] - 1:
            plot_frame(settings, organisms, foods, gen, step)

        # Update fitness function
        for food in foods:
            for org in organisms:
                food_org_dist = dist(org.x, org.y, food.x, food.y)

                # Collision
                if food_org_dist <= 0.075:
                    org.fitness += food.energy
                    food.respawn(settings)

                org.d_food = 100
                org.r_food = 0

        for food in foods:
            for org in organisms:

                food_org_dist = dist(org.x, org.y, food.x, food.y)

                if food_org_dist < org.d_food:
                    org.d_food = food_org_dist
                    org.r_food = calc_heading(org, food)

        for org in organisms:
            org.think()

        # Update organism
        for org in organisms:
            org.update_r(settings)
            org.update_v(settings)
            org.update_pos(settings)

    return organisms

In [56]:
def run(settings):

    # Populate food
    foods = []
    for i in range(0, settings["food_num"]):
        foods.append(food(settings))

    # Populate organisms
    organisms = []
    for i in range(settings["pop_size"]):
        wih_init = np.random.uniform(
            -1, 1, (settings["hnodes"], settings["inodes"])
        )  # mlp weights (input -> hidden)
        who_init = np.random.uniform(
            -1, 1, (settings["onodes"], settings["hnodes"])
        )  # mlp weights (hidden -> output)

        organisms.append(
            organism(settings, wih_init, who_init, name="gen[x]-org[" + str(i) + "]")
        )

    # Cycle through gens
    for gen in range(settings["gens"]):

        # SIMULATE
        organisms = simulate(settings, organisms, foods, gen)

        # EVOLVE
        organisms, stats = evolve(settings, organisms, gen)
        print(
            f'> GEN: {gen}; BEST: {stats["BEST"]}, AVG: {stats["AVG"]}, WORST: {stats["WORST"]}'
        )

In [71]:
run(settings)

> GEN: 0; BEST: 219, AVG: 35.66, WORST: 2
> GEN: 1; BEST: 224, AVG: 74.36, WORST: 21
> GEN: 2; BEST: 167, AVG: 61.24, WORST: 22
> GEN: 3; BEST: 208, AVG: 86.92, WORST: 19
> GEN: 4; BEST: 210, AVG: 86.5, WORST: 12
> GEN: 5; BEST: 208, AVG: 89.9, WORST: 12
> GEN: 6; BEST: 241, AVG: 94.78, WORST: 16
> GEN: 7; BEST: 215, AVG: 94.84, WORST: 10
> GEN: 8; BEST: 225, AVG: 93.9, WORST: 14
> GEN: 9; BEST: 196, AVG: 90.2, WORST: 11
> GEN: 10; BEST: 260, AVG: 93.32, WORST: 12
> GEN: 11; BEST: 173, AVG: 86.82, WORST: 14
> GEN: 12; BEST: 184, AVG: 82.96, WORST: 18
> GEN: 13; BEST: 209, AVG: 82.64, WORST: 2
> GEN: 14; BEST: 225, AVG: 83.38, WORST: 18
> GEN: 15; BEST: 183, AVG: 80.6, WORST: 14
> GEN: 16; BEST: 207, AVG: 85.06, WORST: 12
> GEN: 17; BEST: 175, AVG: 88.22, WORST: 10
> GEN: 18; BEST: 227, AVG: 91.42, WORST: 18
> GEN: 19; BEST: 186, AVG: 88.24, WORST: 11
> GEN: 20; BEST: 224, AVG: 90.82, WORST: 16
> GEN: 21; BEST: 257, AVG: 87.22, WORST: 18
> GEN: 22; BEST: 197, AVG: 88.26, WORST: 14
> GEN

KeyboardInterrupt: 